# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()

,City Name,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,rypefjord,847634.0,70.63,23.67,19.40,73.0,100.0,32.21
1,tiksi,2015306.0,71.69,128.87,-26.37,85.0,0.0,5.39
2,ventosa,2261732.0,39.13,-9.08,60.01,93.0,20.0,2.24
3,georgetown,1735106.0,5.41,100.34,80.60,74.0,20.0,3.36
4,mataura,6201424.0,-46.19,168.86,59.00,91.0,100.0,8.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
gmaps.configure(api_key=g_key)

locations = cities_df[["Latitude", "Longitude"]].astype(float)
humidity = cities_df["Humidity"].astype(float)


In [29]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [51]:
formatted_cities = cities_df.dropna()

narrowed_city_df = cities_df.loc[((formatted_cities["Max Temperature"]<95) & (formatted_cities["Max Temperature"]>70)) & (formatted_cities["Wind Speed"]<=10) & \
                                (formatted_cities["Cloudiness"]<50)]

narrowed_city_df.head()

,City Name,City ID,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
3,georgetown,1735106.0,5.41,100.34,80.60,74.0,20.0,3.36
6,hermanus,3366880.0,-34.42,19.23,72.00,87.0,31.0,8.99
19,necochea,3430443.0,-38.55,-58.74,82.99,49.0,38.0,10.00
24,san vicente,3428068.0,-26.62,-54.13,75.00,99.0,34.0,2.98
38,lompoc,5367788.0,34.64,-120.46,78.80,29.0,1.0,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [57]:
hotel_df = pd.DataFrame(narrowed_city_df, columns=["City Name", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df

,City Name,Latitude,Longitude,Hotel Name
3,georgetown,5.41,100.34,
6,hermanus,-34.42,19.23,
19,necochea,-38.55,-58.74,
24,san vicente,-26.62,-54.13,
38,lompoc,34.64,-120.46,
...,...,...,...,...
491,touros,-5.20,-35.46,
509,san jose de rio tinto,14.93,-85.70,
513,grande riviere du nord,19.58,-72.18,
522,carrefour,18.54,-72.40,


In [63]:
target_coordinates = f"{hotel_df['Latitude']},{hotel_df['Longitude']}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"


params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


response = requests.get(base_url, params=params)

In [62]:
hotel_df['Hotel Name']= hotel_name

hotel_df.to_csv('hotel_export.csv')

In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map